# 0.0 Divide your data into distributed train, val and test sets

Our file todo.txt contains all the images and its corresponding classes. Now we want to split our images into train, val and test files, so that the classes are well distributed (e.g. if the dataset is highly imbalanced the images are not randomly separated).

In [6]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd

# import imgclas
from imgclas import paths, config
# from imgclas.data_utils import load_image, load_data_splits, augment, load_class_names

from sklearn.model_selection import train_test_split

CONF = config.get_conf_dict()

# Customize your image folder if needed
# CONF['general']['images_directory'] = '/media/ignacio/Datos/datasets/semillas/datasets'  # absolute path to image_folder

# splits_dir = paths.get_splits_dir()
splits_dir = '/home/jovyan/image-classification-tf/data/dataset_files/'

# Load the data
data = pd.read_csv(splits_dir+'patientsAll_Corrected.txt', sep="*", header=None, names=['image', 'clase'])

In [7]:
data

,image,clase
0,/home/jovyan/datos/GrupoA_Revision_BC/10527/10...,0
1,/home/jovyan/datos/GrupoA_Revision_BC/10489/10...,0
2,/home/jovyan/datos/GrupoA_Revision_BC/8139/813...,0
3,/home/jovyan/datos/GrupoA_Revision_BC/7203/720...,0
4,/home/jovyan/datos/GrupoA_Revision_BC/7393/739...,0
...,...,...
95,/home/jovyan/datos/Tortuosos/10310/10310_SPIDE...,1
96,/home/jovyan/datos/Tortuosos/9546/9546_SPIDER_...,1
97,/home/jovyan/datos/Tortuosos/1805/1805_spider_...,1
98,/home/jovyan/datos/Tortuosos/3824/3824_spider_...,1


In [8]:
# https://machisnelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/
from collections import Counter
print(Counter(data.clase))

Counter({0: 59, 1: 41})


In [19]:
# split into train test sets
X_train, X_2, y_train, y_2 = train_test_split(data.image, data.clase, test_size=0.1, random_state=1, stratify=data.clase)
print(Counter(y_train))
print(Counter(y_2))

Counter({0: 53, 1: 37})
Counter({0: 6, 1: 4})


In [20]:
X_test, X_val, y_test, y_val = train_test_split(X_2, y_2, test_size=0.5, random_state=1, stratify=y_2)
print(Counter(y_test))
print(Counter(y_val))

Counter({0: 3, 1: 2})
Counter({0: 3, 1: 2})


In [21]:
train=pd.DataFrame(data={'image': X_train, 'clase': y_train})
train.to_csv(r'//home/jovyan/image-classification-tf/data/dataset_files/train.txt', sep=' ', index=None, header=None, mode='a')

In [22]:
val=pd.DataFrame(data={'image': X_val, 'clase': y_val})
val.to_csv(r'/home/jovyan/image-classification-tf/data/dataset_files/val.txt', sep=' ', index=None, header=None, mode='a')

In [23]:
test=pd.DataFrame(data={'image': X_test, 'clase': y_test})
test.to_csv(r'/home/jovyan/image-classification-tf/data/dataset_files/test.txt', sep=' ', index=None, header=None, mode='a')

In [24]:
len(train)

90

In [25]:
len(test)

5

In [26]:
len(val)

5